In [78]:
import tensorly as tl
import numpy as np
import QuantLib as ql
import matplotlib.pyplot as plt
import pysabr

In [79]:
today = ql.Date(1,1,2022)
ql.Settings.instance().setEvaluationDate(today)
day_counter = ql.Actual365Fixed()

In [141]:
def fdm(strike, forward, expiry_time, alpha, beta, nu, rho, t_grid=20, f_grid=200, x_grid=25, scaling_factor=1.0, eps=.0001):
    maturity_date = today + ql.Period(int(365*expiry_time), ql.Days)
    r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, day_counter))
    vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), 0.2, day_counter))
    bs_process = ql.GeneralizedBlackScholesProcess(ql.QuoteHandle(ql.SimpleQuote(forward)), r_ts, r_ts, vol_ts)
    option = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), ql.EuropeanExercise(maturity_date))
    option.setPricingEngine(ql.FdSabrVanillaEngine(forward, alpha, beta, nu, rho, r_ts, t_grid, f_grid, x_grid, 0, scaling_factor, eps))
    return option.impliedVolatility(option.NPV(), bs_process, 1.0e-7, 1000, -10.0, 100.0)

In [144]:
def fdm_price(strike, forward, expiry_time, alpha, beta, nu, rho, t_grid=20, f_grid=200, x_grid=25, scaling_factor=1.0, eps=.0001):
    maturity_date = today + ql.Period(int(365*expiry_time), ql.Days)
    r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, day_counter))
    option = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), ql.EuropeanExercise(maturity_date))
    option.setPricingEngine(ql.FdSabrVanillaEngine(forward, alpha, beta, nu, rho, r_ts, t_grid, f_grid, x_grid, 0, scaling_factor, eps))
    return option.NPV()

In [142]:
strikes = np.exp(np.linspace(-1, 1, 10))
forward = 1
expiry_time = 0.5
alpha = 0.2
beta = 0.4
nu = 0.5
rho = -0.4
# hagan_vols = [ql.sabrVolatility(strike, forward, day_counter.yearFraction(today, today + ql.Period(int(365*expiry_time), ql.Days)), alpha, beta, nu, rho) for strike in strikes]
# fdm_vols = [fdm(strike, forward, expiry_time, alpha, beta, nu, rho, scaling_factor=10) for strike in strikes]

In [143]:
# plt.plot(np.log(strikes), np.array(hagan_vols)-np.array(fdm_vols))
# plt.plot(np.log(strikes), fdm_vols)
# plt.plot(np.log(strikes), hagan_vols)
plt.plot(np.log(strikes), [ql.sabrVolatility(strike, forward, day_counter.yearFraction(today, today + ql.Period(int(365*expiry_time), ql.Days)), alpha, beta, nu, rho) - fdm(strike, forward, expiry_time, alpha, beta, nu, rho) for strike in strikes])
plt.plot(np.log(strikes), [ql.sabrVolatility(strike, forward, day_counter.yearFraction(today, today + ql.Period(int(365*expiry_time), ql.Days)), alpha, beta, nu, rho) - fdm(strike, forward, expiry_time, alpha, beta, nu, rho, 60, 600, 75) for strike in strikes])
# plt.plot(np.log(strikes), [ql.sabrVolatility(strike, forward, day_counter.yearFraction(today, today + ql.Period(int(365*expiry_time), ql.Days)), alpha, beta, nu, rho) - fdm(strike, forward, expiry_time, alpha, beta, nu, rho, 120, 1200, 150) for strike in strikes])

RuntimeError: root not bracketed: f[-10,100] -> [3.585780e-01,3.588210e-01]

In [91]:
fdm(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166)

1.1187952607144929

In [34]:
fdm(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166, scaling_factor=0.25)

0.9787179973136005

In [5]:
fdm(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166, 40, 400, 50)

1.1181181462752756

In [6]:
fdm(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166, 60, 600, 75)

1.117992671559152

In [92]:
fdm(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166, 200, 2000, 250)

1.1179012631386676

In [100]:
fdm(0.8, 1.0, 0.2, 0.3, 0.4, 0.5, -0.6, 200, 2000, 250, eps=.0000001)

0.35606294808831723

In [ ]:
plt.plot([fdm(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166, i*20, i*200, i*25) for i in range(2)])

In [7]:
ql.sabrVolatility(0.466, 1.0, 0.6, 0.825, 0.299, 0.41, -0.166)

1.1253570466015932

In [102]:
ql.sabrVolatility(0.8, 1.0, 0.2, 0.3, 0.4, 0.5, -0.6)

0.3562478823424832

In [147]:
fdm_price(0.466, 1.0, 0.6, 0.825, 0.299, 0.410, -0.166, eps=1.0e-7)

0.5745809606064761

In [146]:
fdm_price(0.466, 1.0, 0.6, 0.825, 0.299, 0.410, -0.166, 200, 2000, 250)

0.5744852218698189